### ЦЕЛЬ
Подготовить основу рекомендательной системы в онлайн-школе MasterMind.

### ЗАДАЧИ

Подготовка рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

Изначальным воплощением этой системы может стать таблица, в которой курсам будет соответствовать по две рекомендации.

### КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

- Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
- Обработать данные средствами Python.
- Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.

#### 1. Знакомство с данными

В распоряжении две таблицы:

- carts - с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);
- cart items с данными о курсах, которые пользователи добавили в корзину.

Необходимые таблицы находятся в схеме final в Metabase.
Итак (посредством SQL удалось выяснить):

- В данных находятся продажи за 2017 и 2018 года.
- 49006 клиентов покупали курсы
- всего есть 127 различных курсов 
- среднее число купленных курсов на одного клиента: 1.44
- 12656 клиентов купили больше одного курса.

Подготовим SQL-запрос для выгрузки данных по продажам курсов в разрезе пользователей (купивших более одного курса):

In [1]:
# WITH users AS (
# select 
#       count(distinct cri.resource_id) as cnt
#     , cr.user_id
# from final.carts cr
# left join final.cart_items cri on cr.ID = cri.cart_id
# where cr.state = 'successful'
# and cri.resource_type = 'Course'
# group by cr.user_id
# )
# select 
#       cr.user_id
#     , cr.created_at as "cart_created_at"
#     , cr.id as "cart_id"
#     , cr.promo_code_id
#     , cr.purchased_at
#     , cr.updated_at as "cart_updated_at"
#     , cri.id as "cart_item_id"
#     , cri.resource_id as "course_id"
#     , cri.created_at as "cartItem_created_at"
#     , cri.updated_at as "cartItem_updated_at"
# from 
#    final.carts cr
# left join 
#    final.cart_items cri on cr.ID = cri.cart_id
# join 
#    users us on us.user_id = cr.user_id
#             and us.cnt > 1
# where 
#    cr.state = 'successful'
# and cri.resource_type = 'Course'

#### 2. Обработка данных

Перейдём к анализу полученного списка с помощью Python.

In [2]:
import pandas as pd
df = pd.read_csv(r'''E:\DOKU\Аналитика данных\Python\VS Code\data_base\query_result.csv''')

In [3]:
df.head()

,user_id,cart_created_at,cart_id,promo_code_id,purchased_at,cart_updated_at,cart_item_id,course_id,cartItem_created_at,cartItem_updated_at
0,1010882,2017-01-19T07:50:41.298,230789,NaN,2017-01-19T07:52:08.59,2017-01-19T07:52:08.586,522159,490,2017-01-19T07:50:41.313,2017-01-19T07:50:41.313
1,906674,2016-10-24T15:49:18.112,189760,NaN,2017-01-11T12:20:13.399,2017-01-11T12:20:13.396,460355,357,2016-10-24T15:49:18.127,2016-10-24T15:49:18.127
2,160494,2017-04-03T09:03:37.633,258743,3548100.0,2017-06-15T18:31:46.577,2017-06-15T18:31:46.573,575400,507,2017-04-03T09:05:18.837,2017-04-03T09:05:18.837
3,749529,2017-01-18T22:50:51.644,230716,NaN,2017-01-18T22:54:15.901,2017-01-18T22:54:15.896,522058,489,2017-01-18T22:50:51.658,2017-01-18T22:50:51.658
4,1010802,2017-01-19T00:24:25.214,230722,NaN,2017-01-19T00:26:00.862,2017-01-19T00:26:00.858,522063,514,2017-01-19T00:24:25.227,2017-01-19T00:24:25.227


Разобьём все покупки курсов по парам, после чего ранжируем их для каждого курса.

In [4]:
import numpy as np
# для начала сгруппируем курсы по пользователям
# метод np.unique() возвращает уникальные и отсортированные значения

group_df = df.groupby('user_id')['course_id'].apply(lambda x:list(np.unique(x))).reset_index()
group_df.head()

,user_id,course_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [5]:
# Далее разобьем все покупки курсов по парам.

import itertools
# здесь функция itertools.combinations() принимает два параметра: итерируемоя последовательность и длина возвращаемых кортежей.
# Возвращаемое значение - итератор со всеми возможными комбинациями элементов входной последовательности.

list_courses = list()
for course in group_df['course_id']: # здесь course - список курсов для текущего пользователя 
    for pair in itertools.combinations(course, 2): # здесь pair - текущая комбинация (пара) из этого списка
        list_courses.append(pair)
        
# --> [(516, 1099), (356, 357), (356, 1125), (357, 1125), (553, 1147), (361, 1138), (356, 357)..]

In [6]:
# количество всех пар
len(list_courses)

40017

In [7]:
# Сколько различных пар курсов встречаются в покупках клиентов:
from collections import Counter

# класс collections.Counter() предназначен для подсчета количества повторений элементов в последовательности.
# Это подкласс словаря dict, в которой элементы хранятся в виде словарных ключей, а их счетчики хранятся в виде значений словаря.
cnt = Counter(list_courses)
len_of_dict = len(dict(cnt))
len_of_dict

# --> {(516, 1099): 25, (356, 357): 100, (356, 1125): 44, ...}
# длина словаря - это и есть количество уникальных пар

3989

In [8]:
# Cамая популярная пара курсов.
# Метод Counter.most_common() возвращает список из N наиболее распространенных элементов и их количество от наиболее распространенных до наименее.
Counter(list_courses).most_common(1)

[((551, 566), 797)]

#### 3. Составление итоговой таблицы с рекомендациями

Итоговая таблица будет состоять из трёх столбцов:

- Курс, к которому идёт рекомендация.
- Курс для рекомендации № 1 (самый популярный).
- Курс для рекомендации № 2 (второй по популярности).

In [9]:
# И-так, сохраним уникальные пары и их количество в словарь. Они будут отсортированы от наиболее распространенных до наименее. 
# Словарь получится такого вида:
# {(551, 566): 797,
# (515, 551): 417,
# (489, 551): 311,
# (523, 551): 304,
# (566, 794): 290, ...}

sorted_pairs = dict(Counter(list_courses).most_common(len_of_dict))

Условимся, что будем делать, если одна из рекомендаций встречается слишком редко:
- Прежде всего нужно установить минимальную границу — какое количество раз считать слишком малым.
- Вместо малопопулярного курса выведем курс, который подходит лучше.

In [10]:
# Выведем количество пар курсов из нашего словаря в отдельный список:
count_list = list(sorted_pairs.values())

# --> [797, 417, 311, ...]

In [11]:
# найдём 50% процентиль (медиана) для данной выборки. Т.е. функция должна вернуть такое значение, 
# которое меньше ровно половины элементов массива:

percentile = np.percentile(count_list, 50)
percentile

3.0

In [12]:
# удаляем из словаря те пары, для которых количество меньше или равно процентиля.

not_popular_course = []

for pair in sorted_pairs.keys():
    if sorted_pairs.get(pair) <= percentile:
        not_popular_course.append(pair) # список непопулярных пар

for pair in not_popular_course:
    sorted_pairs.pop(pair)
    
len(sorted_pairs)

1860

In [13]:
# Создадим функцию, которая на вход получает один текущий уникальный курс.
# Для каждого курса мы получим два отсортированных кортежа, где вторыми элементами будут являться наши рекомендации.

def recommendations(course):
    local_list = []
    
    # обойдём наш словарь, где ключ (i) - кортеж (пара)
    for i in sorted_pairs.keys():
        if i[0] == course: # если первый элемент этой пары - наш курс, 
                           # добавим в конец списка кортеж вида [(551, 566), (551, 552) ...]
            local_list.append(i)
    return local_list[:2]

In [14]:
# создадим список уникальных значений курсов
list_courses_unique = df.course_id.unique();

In [15]:
# выделим наиболее популяпную пару, которая будет заменять малопопулярные курсы
most_popular_course_id = list(sorted_pairs.keys())[0]
most_popular_course_id

(551, 566)

In [16]:
# Построим таблицу рекомендаций
df_recommend = pd.DataFrame(columns = ['First_recommendation', 'Second_recommendation'])

# цикл по каждому уникальному курсу
for i in list_courses_unique:
    local_reccommend = recommendations(i)
    
    if len(local_reccommend) == 2: # если функция возвращает два кортежа
        df_recommend.loc[i] = [local_reccommend[0][1], local_reccommend[1][1]]
    elif len(local_reccommend) == 1:
        df_recommend.loc[i] = [local_reccommend[0][1], most_popular_course_id[0]]
    else:
        df_recommend.loc[i] = [most_popular_course_id[0], most_popular_course_id[1]]

df_recommend.head(10)

,First_recommendation,Second_recommendation
490,566,551
357,571,1125
507,570,752
489,551,515
514,551,515
552,564,566
515,551,523
523,551,552
569,572,840
363,511,562


И-так, мы получили рекоммендации для каждого курса.
Так же на одном примере посмотрим, как заполнилась таблица в случае, если мало продаж.
Наприме, для курса № 1147 функция возвращает только один кортеж (одну пару):

In [17]:
recommendations(1147)

[(1147, 1187)]

Выведем значения таблицы для этого курса:

In [18]:
df_recommend.loc[1147].head()

First_recommendation     1187
Second_recommendation     551
Name: 1147, dtype: int64

Мы видим, что первая рекомендация 1187 взята из вернувшейся пары, а вторая - это значение из самой популярной пары.